In [22]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [23]:
dataset=pd.read_csv('Churn_Modelling.csv')

In [24]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [25]:
X=dataset.iloc[:,3:13]
y=dataset.iloc[:,13]

In [26]:
X.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10


In [27]:
#Create Dummy values for categorical features
geography=pd.get_dummies(X["Geography"],drop_first=True)
gender=pd.get_dummies(X["Gender"],drop_first=True)

In [28]:
#Concat The dataframes
X=pd.concat([X,geography,gender],axis=1)

In [29]:
#Drop uneccesary columns
X=X.drop(['Geography','Gender'],axis=1)

In [30]:
X.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [31]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

In [32]:
#Feature Scaling

from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

In [33]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [34]:
from keras.models import Sequential
from keras.layers import Dense, Dropout , Activation , Embedding , LeakyReLU, Flatten , BatchNormalization
from keras.activations import relu,sigmoid

In [35]:
def create_model(layers,activation):
    model=Sequential()
    for i,nodes in enumerate(layers):
        if(i==0):
            
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
            model.add(Dropout(0.3))
    model.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
    return model

In [36]:
model=KerasClassifier(build_fn=create_model,verbose=0)

C:\Users\harir\AppData\Local\Temp\ipykernel_17600\1314563489.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=create_model,verbose=0)


In [37]:
layers=[[20],[40,20],[45,30,15]]
activations=['sigmoid','relu']
param_grid=dict(layers=layers,activation=activations,batch_size=[128,256],epochs=[30])
grid=GridSearchCV(estimator=model,param_grid=param_grid,cv=5)


In [38]:
grid_result=grid.fit(X_train,y_train)


In [39]:
print(grid_result.best_score_,grid_result.best_params_)

0.85625 {'activation': 'relu', 'batch_size': 256, 'epochs': 30, 'layers': [45, 30, 15]}


In [40]:
y_pred=grid.predict(X_test)
y_pred=(y_pred>0.5)

63/63 [==============================] - 0s 3ms/step
